In [ ]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]

# 🤖 Reinforcement Learning

TODO description

Kickstart the simulation.

In [ ]:
from controllables.energyplus import (
    World,
    #WeatherModel,
    #Report,
    Actuator,
    OutputVariable,
)

from energyplus.dataset.basic import dataset as _epds_

world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.Specs.Output(
        report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    runtime=World.Specs.Runtime(
        recurring=True,
        #design_day=True,
    ),
)

Configure the controller (an RLlib algorithm).

In [ ]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from controllables.energyplus import (
    Actuator,
    OutputVariable,
)
from controllables.core.tools.gymnasium import (
    VariableBox,
)
from controllables.core.tools.ray import ExternalEnv

from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray.rllib.algorithms.callbacks import DefaultCallbacks


config = (
    PPOConfig()
    .environment(
        ExternalEnv, 
        env_config=ExternalEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'thermostat': VariableBox(
                    low=15., high=20.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Heating Setpoint',
                    key='CORE_MID',
                ))
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='People Air Temperature',
                    key='CORE_MID',
                )),
            }),
            system=lambda: world,
            reward_function=lambda agent: 1,
            episode_events={
                'step': 'begin_zone_timestep_after_init_heat_balance',
            },
        )
    )
    .rollouts(
        create_env_on_local_worker=True,
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework('torch')
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)

algo = PPO(
    config=config,
)

Train the algorithm.

In [ ]:
world.awaitable.run()

Exception ignored on calling ctypes callback function: <function EventManager._core_callback_setters.<locals>._Dispatcher._state at 0x7fc9bc6ea840>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 212, in cb_
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 209, in cb_
    return cb(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 200, in _state
    self._manager.__call__(
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 291, in __call__
    return super().__call__(
           ^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/controllables/core/specs/callbacks.py", line 563, in __call__
    return self[ref].__call__(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/

In [ ]:
# TODO
#algo.workers.local_worker().env._engine

In [ ]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import MessageLogger

from controllables.energyplus.logging import ProgressLogger

world.add(
    ProgressLogger(),
)

In [ ]:
# start training
for _ in range(5):
    print(algo.train())

Evaluate the algorithm.

In [ ]:
algo.evaluate()

Stop the simulation environment when we are done!

In [ ]:
# TODO tree view
import pandas as _pandas_
import itables as _itables_
_itables_.init_notebook_mode(all_interactive=True)

_pandas_.DataFrame(world.variables.available_keys().group(type)[Actuator.Ref])

In [ ]:
world.stop()